In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [19]:
unique_labels = ['hate', 'normal', 'offensive']

# Create label mappings
label_map = {label: idx for idx, label in enumerate(unique_labels)}
reverse_label_map = {idx: label for label, idx in label_map.items()}

print(f"Unique labels: {unique_labels}")
print(f"Label mapping: {label_map}")

df_shuffled = pd.read_csv('datasets/hindi/hasoc_hindi_dataset/hindi_preprocessed_data.csv')


# Split the data
train_df, temp_df = train_test_split(
    df_shuffled,
    test_size=0.3,
    random_state=42,
    stratify=df_shuffled['label_encoded']
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df['label_encoded']
)

print(f"\nDataset splits:")
print(f"Train: {len(train_df)}")
print(f"Validation: {len(val_df)}")
print(f"Test: {len(test_df)}")

# ============ MODEL SETUP ============

MODEL_NAME = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Dataset class
class HindiMultiClassDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
train_dataset = HindiMultiClassDataset(
    train_df['clean_text'].values,
    train_df['label_encoded'].values,
    tokenizer
)

val_dataset = HindiMultiClassDataset(
    val_df['clean_text'].values,
    val_df['label_encoded'].values,
    tokenizer
)

test_dataset = HindiMultiClassDataset(
    test_df['clean_text'].values,
    test_df['label_encoded'].values,
    tokenizer
)

# ============ COMPUTE METRICS FUNCTION ============

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    # Weighted metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)

    # Per-class metrics
    precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(
        labels, predictions, average=None, labels=list(range(len(unique_labels)))
    )

    metrics = {
        'accuracy': acc,
        'f1_weighted': f1,
        'precision_weighted': precision,
        'recall_weighted': recall,
    }

    # Add per-class metrics
    for i, label_name in reverse_label_map.items():
        metrics[f'f1_{label_name}'] = f1_per_class[i]
        metrics[f'precision_{label_name}'] = precision_per_class[i]
        metrics[f'recall_{label_name}'] = recall_per_class[i]

    return metrics

# ============ INITIALIZE MODEL ============

num_classes = len(unique_labels)
print(f"\nTraining model with {num_classes} classes: {unique_labels}")

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_classes,
    id2label=reverse_label_map,
    label2id=label_map
)

# ============ TRAINING ARGUMENTS ============

training_args = TrainingArguments(
    output_dir='./hindi_results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./hindi_logs',
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=10,
    save_strategy='steps',
    save_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='f1_weighted',
    greater_is_better=True,
    learning_rate=2e-5,
    fp16=False,
    dataloader_pin_memory=False,
    report_to=None,
)

# ============ INITIALIZE TRAINER ============

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# ============ TRAIN THE MODEL ============

print(f"\nStarting training for Hindi {num_classes}-class model...")
trainer.train()

# Save the model
model_save_path = f"hindi_hate_speech_detector"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nModel saved to: {model_save_path}")

# ============ EVALUATE THE MODEL ============

test_results = trainer.evaluate(test_dataset)
print(f"\n=== TEST RESULTS ===")
for key, value in test_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")

# Detailed predictions
test_predictions = trainer.predict(test_dataset)
y_pred = np.argmax(test_predictions.predictions, axis=1)
y_true = test_predictions.label_ids

# Classification report
print(f"\n=== DETAILED CLASSIFICATION REPORT ===")
target_names = [reverse_label_map[i] for i in sorted(reverse_label_map.keys())]
print(classification_report(y_true, y_pred, target_names=target_names))

# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names,
            yticklabels=target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix - Hindi Hate Speech Detection')
plt.tight_layout()
plt.show()

print(f"\nClass Mapping:")
for idx, name in enumerate(target_names):
    print(f"  {idx}: {name}")

# ============ PREDICTION FUNCTION ============

def predict_hindi_text(text, model_path=model_save_path):
    """
    Predict class for new Hindi text
    """
    # Load model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    # Clean text
    cleaned_text = clean_text(text)
    
    # Tokenize
    inputs = tokenizer(
        cleaned_text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
    
    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Get predicted class
    predicted_class_id = predictions.argmax().item()
    predicted_class = reverse_label_map[predicted_class_id]
    confidence = predictions[0][predicted_class_id].item()
    
    # Get probabilities for all classes
    class_probabilities = {
        reverse_label_map[i]: predictions[0][i].item() 
        for i in range(len(unique_labels))
    }
    
    return {
        'text': text,
        'cleaned_text': cleaned_text,
        'predicted_class': predicted_class,
        'confidence': confidence,
        'probabilities': class_probabilities
    }

# Test predictions
print("\n=== TESTING PREDICTIONS ===")
test_texts = [
    "बांग्लादेश की शानदार वापसी, भारत को 314 रन पर रोका",
    "तुम जैसे हरामियों के लिए बस जूतों की कमी है",
    "#कांग्रेस के इस #कमीने की #करतूत को देखिए",
    "मुंबई में बारिश से लोगों को काफी समस्या हो रही है",
]

for text in test_texts:
    result = predict_hindi_text(text)
    print(f"\nText: {text[:50]}...")
    print(f"Predicted: {result['predicted_class']} (Confidence: {result['confidence']:.4f})")
    print(f"Probabilities: {result['probabilities']}")

print("\n=== CODE EXECUTION COMPLETE ===")

Unique labels: ['hate', 'normal', 'offensive']
Label mapping: {'hate': 0, 'normal': 1, 'offensive': 2}

Dataset splits:
Train: 4186
Validation: 897
Test: 898

Training model with 3 classes: ['hate', 'normal', 'offensive']


Cancellation requested; stopping current tasks.


KeyboardInterrupt: 